In [ ]:
%matplotlib inline
import os
import sys
import numpy as np
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from models import MTC

In [ ]:
datasets = ['aotm2011', '30music']
metrics = ['R-Precision', 'Hit-Rate']
TOPs = [5, 10, 20, 30, 50, 100, 200, 300, 500, 1000]

In [ ]:
dix = 1
task = 2
dataset_name = datasets[dix]
data_dir = 'data/%s/setting%d' % (dataset_name, task)

In [ ]:
algo_names = ['Logistic Regression', 'Popularity Rank', 'Multitask Classification']
algos = ['br2', 'pop', 'pla']

In [ ]:
#col_indices = pd.MultiIndex.from_product([algo_names, ['K=%d' % k for k in range(1,5)]])
rows = ['K=%d' % k for k in range(1,5)]

In [ ]:
df = pd.DataFrame(index=rows, columns=algo_names)

In [ ]:
metric = metrics[0]

In [ ]:
for j in range(len(algos)):
    algo_name = algo_names[j]
    algo = algos[j]
    for k in range(1,5):
        fperf = os.path.join(data_dir, 'perf-%s-%d.pkl' % (algo, k))
        if not os.path.exists(fperf):
            continue
        perf_dict = pkl.load(open(fperf, 'rb'))
        #print(perf_dict)
        df.loc['K=%d' % k, algo_name] = perf_dict[dataset_name]['Test'][metric] * 1000

In [ ]:
df

In [ ]:
tab_str = df.to_latex(float_format=lambda x: '$%.1f$' % x, na_rep='N/A', \
                      column_format='l|*{%d}{c}' % len(algos), escape=False)
                      #multicolumn=True, multicolumn_format='c', escape=False)

In [ ]:
print('\\begin{table}[!h]')
print('\centering')
print('\\caption{R-Precision $\\times 10^3$ (%s)}' % dataset_name)
print('\\label{tab:hitrate_pla_30music}')    
print(tab_str)
print('\\end{table}')

In [ ]:
metric = metrics[1]

In [ ]:
nrows, ncols = 2, 2
colors = ['r', 'g', 'b']
fig = plt.figure(figsize=[20, 10])
for k in range(1,5):
    fperfs = [os.path.join(data_dir, 'perf-%s-%d.pkl' % (algo, k)) for algo in algos]
    perf_dicts = [pkl.load(open(fperf, 'rb')) for fperf in fperfs]
    for j in range(len(perf_dicts)):
        x = TOPs
        y = [perf_dicts[j][dataset_name]['Test'][metric][k] for k in TOPs]
        plt.subplot(nrows, ncols, k)
        plt.plot(x, y, ls='-', c=colors[j], label=algo_names[j])
        plt.title('K=%d' % k)
    plt.legend(loc='best')
    if k > 2:
        plt.xlabel('Number of recommendations')
    if k % 2 == 1:
        plt.ylabel('Hit rate')
#plt.suptitle('Hit rates of playlist augmentation (%s)' % dataset_name, fontsize=12)
plt.savefig('%s-%d.svg' % (dataset_name, task))